In [196]:
import psycopg2
import json
import pandas as pd   #to create dataframe
import requests       #to send request to the URL
from bs4 import BeautifulSoup #to get the content in the form of HTML

#establishing the connection
conn = psycopg2.connect(
   database="data_tvseries", user='rana', password='password', host='localhost', port= '5432'
)

#Creating a cursor object using the cursor() method
cursor = conn.cursor()


In [197]:
#create table queries
sql_stud = '''CREATE TABLE studios (
studio_name VARCHAR(255),
year_created INT,
address VARCHAR(255),
PRIMARY KEY (studio_name))'''

sql_users = '''CREATE TABLE users (
user_id char(8),
username VARCHAR(50),
first_name VARCHAR(255),
last_name VARCHAR(255),
PRIMARY KEY (user_id))'''

sql_casts = '''CREATE TABLE casts (                                                 
actor_names varchar(255),
genders varchar(50),
nationalities varchar(50),
birth_dates date,
PRIMARY KEY (actor_names))'''

sql_chars = '''CREATE TABLE character (                                                 
char_names varchar(255),
role varchar(50),
actor_names varchar(255),
PRIMARY KEY(char_names),
FOREIGN KEY(actor_names) references casts(actor_names))'''

sql_tvseries = '''CREATE TABLE tv_series(                                                 
series_title varchar(255) ,
start_year char(10) ,
end_year char(10) ,
seasons int,
rank int,
runtime varchar(50),
finalrate float,
director varchar(255) ,
status varchar(50) ,
description varchar(300),
genres varchar(255) ,
studio_name varchar(255),
char_names varchar(255),
pRIMARY KEY(series_title),
FOREIGN KEY(studio_name) REFERENCES studios(studio_name), 
FOREIGN KEY(char_names) REFERENCES character(char_names));'''

sql_episodes = '''CREATE TABLE episodes(                                                   
episode_title varchar(255),
air_date date,
description varchar(255),
series_title varchar(255),
PRIMARY KEY(episode_title),
FOREIGN KEY(series_title) REFERENCES tv_series(series_title))'''

sql_rating = '''CREATE TABLE rating(
user_id char(8),
series_title varchar(255),
rating float,
comment varchar(255),primary key(user_id,series_title),
FOREIGN KEY (user_id) REFERENCES users(user_id), FOREIGN KEY (series_title) references tv_series(series_title))'''


In [198]:
#executing queries
cursor.execute(sql_stud)
cursor.execute(sql_users)
cursor.execute(sql_casts)
cursor.execute(sql_chars)
cursor.execute(sql_tvseries)
cursor.execute(sql_episodes)
cursor.execute(sql_rating)

In [199]:
with open('/Users/rana/Downloads/finaldata.json') as docs:
        records = json.load(docs)

        for doc in records:
                ranked = doc['Rank']
                tvseries_title = doc['TV Series Title']
                start_y = doc['Year Started']
                end_y = doc['Year Ended']
                runtime = doc['Duration']
                genre = doc['Genres']
                rating = doc['Rating']
                votes = doc['Votes']
                description = doc['Description']
                actors = doc['Actors']

                genre_str = ", ".join(genre)  # Convert the genre list to a string

                # Escape the square brackets in the genre string
                genre_str = genre_str.replace("[", "\\[")
                genre_str = genre_str.replace("]", "\\]")

                actors_str = ", ".join(actors)  # Convert the genre list to a string

                # Escape the square brackets in the genre string
                actors_str = actors_str.replace("[", "\\[")
                actors_str = actors_str.replace("]", "\\]")

                # Construct the SQL statement
                sql_inserttv = "INSERT INTO tv_series(series_title, start_year, end_year, genres, rank, description, runtime, finalrate) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"
                sql_insertactors = "INSERT INTO casts(actor_names) VALUES (%s)"

                cursor.execute(sql_inserttv, (tvseries_title, start_y, end_y, genre_str, ranked, description, runtime, rating))
                cursor.execute(sql_insertactors, (actors_str,))


In [201]:
conn.commit()

cursor.close()
conn.close()